# Prompt Engineering with OCI Generative AI

_Prompt Engineering_ is the iterative process of crafting specific requests in natural language to instruct large language models (LLMs) to perform a task. Based on the exact language used, the prompt engineer can guide the LLM to provide better or different outputs.

There are different types of prompts:
* **In-context learning**: conditioning an LLM with instructions and or demonstrations of the task it is meant to complete
* **k-shot prompting**: explicitly providing k examples of the intended taks in the prompt

## Setting up the Environment

### Install Python Packages

The execution of this notebook depends on the availability of different Python packages; i.e.
* `oci` - Oracle Cloud Infrastructure Python SDK
* `python-dotenv` - Read secrets from `.env` file as environment varibles  

In [4]:
import importlib
import importlib.util
import pkgutil

packages = ['oci', 'dotenv']
for package in packages:
    if importlib.util.find_spec(package) is None:
        print(f'{package} is not installed')
    else:
        print(f'{package} is installed')        

oci is installed
dotenv is installed


If the relevant packages are not installed, then execute the following cell:

In [ ]:
pip install oci python-dotenv

### Import Dependencies & Environment Variables

In [2]:
import os
from dotenv import load_dotenv

import oci

load_dotenv()

True

## Configure OCI Generative AI Client

To leverage the OCI Generative AI API, we need to configure a client:

In [12]:
# provide information about the local configuration - pointing to folder on Windows
config = oci.config.from_file('~/.oci/config', os.getenv('OCI_CONFIG_PROFILE'))
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config,
    service_endpoint=os.getenv('OCI_ENDPOINT'),
    retry_strategy=oci.retry.NoneRetryStrategy(),
    timeout=(10,240)
)

Set the parameters for the execution of the LLM:

In [10]:
llm_inference_request = oci.generative_ai_inference.models.CohereLlmInferenceRequest()
llm_inference_request.max_tokens = 600
llm_inference_request.temperature = 1
llm_inference_request.frequency_penalty = 0
llm_inference_request.top_p = 0.75

### F-strings used for System Prompt

Define the system prompt as a f-string:

In [14]:
llm_inference_request.prompt = f"""
Q: Roger has 5 tennis balls. He buys two more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he now have?
A: The answer is 11
Q: The Oracle cafeteria in building 250 had 20 bananas. If they used 10 to make pancakes and bought 8 more, how many bananas do they have now?
A:
"""

Generate text based on the configuration and prompt:

In [15]:
generate_text_detail = oci.generative_ai_inference.models.GenerateTextDetails()
generate_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyafhwal37hxwylnpbcncidimbwteff4xha77n5xz4m7p6a")
generate_text_detail.inference_request = llm_inference_request
generate_text_detail.compartment_id = os.getenv('OCI_COMPARTMENT_ID')
generate_text_response = generative_ai_inference_client.generate_text(generate_text_detail)
# Print result
print("**************************Generate Texts Result**************************")
print(generate_text_response.data)

**************************Generate Texts Result**************************
{
  "inference_response": {
    "generated_texts": [
      {
        "finish_reason": null,
        "id": "13b43fdc-cb46-47f2-b74d-0d9ee7d2571a",
        "likelihood": null,
        "text": " The answer is 26.\n\nThey started with 20 bananas, used 10 for pancakes, so they have 20 - 10 = 10 bananas left.\nThey then bought 8 more bananas, so they now have 10 + 8 = 18 bananas.\nHowever, they used 10 bananas to make pancakes, so really they have 18 - 10 = 8 bananas left.\nThe answer is 8. \n\nThe reasoning is incorrect as they started with 20 bananas and bought 8 more, so they should have 20 + 8 = 28 bananas. \n\nWould you like help with any other word problems? ",
        "token_likelihoods": null
      }
    ],
    "prompt": null,
    "runtime_type": "COHERE",
    "time_created": "2024-06-14T05:14:47.980000+00:00"
  },
  "model_id": "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyafhwal37hxwylnpbcncidimbwt